In [1]:
import stim
import pymatching
from soft_info import RepCodeIQSimulator


DISTANCE = 7
ROUNDS = 7

after_clifford_depolarization = 8e-2
after_reset_flip_probability = 1e-2
before_measure_flip_probability = 1e-2 
before_round_data_depolarization = 1e-2

# circuit = stim.Circuit.generated("repetition_code:memory",
#                                 distance=DISTANCE,
#                                 rounds=ROUNDS,
#                                 after_clifford_depolarization=after_clifford_depolarization, #two-qubit-fidelity,
#                                 after_reset_flip_probability=after_reset_flip_probability, #reset error,
#                                 before_measure_flip_probability=before_measure_flip_probability, #measurement error,
#                                 before_round_data_depolarization=before_round_data_depolarization) #idle error)


model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)
# print(circuit)


# circuit.to_file("stim")
for i in circuit:
    if i.name == 'MR':
        continue



circuit2 = stim.Circuit.from_file("stim")
print(circuit2)

R 0 1 2 3 4 5 6 7 8 9 10 11 12
X_ERROR(0.01) 0 1 2 3 4 5 6 7 8 9 10 11 12
TICK
DEPOLARIZE1(0.01) 0 2 4 6 8 10 12
CX 0 1 2 3 4 5 6 7 8 9 10 11
DEPOLARIZE2(0.08) 0 1 2 3 4 5 6 7 8 9 10 11
TICK
CX 2 1 4 3 6 5 8 7 10 9 12 11
DEPOLARIZE2(0.08) 2 1 4 3 6 5 8 7 10 9 12 11
TICK
X_ERROR(0.01) 1 3 5 7 9 11
M 1 3 5 7 9 11
X_ERROR(0.01) 1 3 5 7 9 11
DETECTOR(1, 0) rec[-6]
DETECTOR(3, 0) rec[-5]
DETECTOR(5, 0) rec[-4]
DETECTOR(7, 0) rec[-3]
DETECTOR(9, 0) rec[-2]
DETECTOR(11, 0) rec[-1]
REPEAT 6 {
    TICK
    DEPOLARIZE1(0.01) 0 2 4 6 8 10 12
    CX 0 1 2 3 4 5 6 7 8 9 10 11
    DEPOLARIZE2(0.08) 0 1 2 3 4 5 6 7 8 9 10 11
    TICK
    CX 2 1 4 3 6 5 8 7 10 9 12 11
    DEPOLARIZE2(0.08) 2 1 4 3 6 5 8 7 10 9 12 11
    TICK
    X_ERROR(0.01) 1 3 5 7 9 11
    M 1 3 5 7 9 11
    X_ERROR(0.01) 1 3 5 7 9 11
    SHIFT_COORDS(0, 1)
    DETECTOR(1, 0) rec[-6] rec[-12]
    DETECTOR(3, 0) rec[-5] rec[-11]
    DETECTOR(5, 0) rec[-4] rec[-10]
    DETECTOR(7, 0) rec[-3] rec[-9]
    DETECTOR(9, 0) rec[-2] rec[-8]

In [15]:
meas = circuit2.compile_sampler().sample(1000)
meas

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False,  True, False],
       [ True,  True, False, ..., False,  True, False],
       ...,
       [False,  True, False, ..., False,  True, False],
       [False, False,  True, ..., False,  True,  True],
       [False, False, False, ..., False, False, False]])

In [13]:
syndromes = circuit.compile_detector_sampler().sample(1000, append_observables=True)
print(syndromes.shape)
print(syndromes)



(1000, 49)
[[False  True  True ... False False False]
 [False False False ... False False False]
 [ True False  True ... False False  True]
 ...
 [ True False  True ...  True False False]
 [False False False ...  True False  True]
 [False False False ... False False False]]


In [61]:
circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=after_clifford_depolarization, #two-qubit-fidelity,
                                after_reset_flip_probability=after_reset_flip_probability, #reset error,
                                before_measure_flip_probability=1e-8, #measurement error,
                                before_round_data_depolarization=before_round_data_depolarization) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [62]:
num_err = 0

for i in range(len(syndromes)):
    predicted_flip = matching.decode(syndromes[i][:-1])
    if bool(predicted_flip[0]) != (syndromes[i][-1]):

        num_err += 1

print("Error rate:", num_err, "out of:", len(syndromes), "total")  



Error rate: 74 out of: 1000 total
